In [6]:
import pandas as pd
import numpy as np
import pyodbc
import datetime
import os
import re

In [9]:
today=datetime.datetime.now()
file_path="C:\\Users\\bedlamini\\Documents\\VAT Schedules\\Format\\106755134 2023 October 2023 11 26 Transrail ERS VAT schedule October 2023.xlsx"
PTIN = re.search(r"\d{9}", "106755134 2023 October 2023 11 26 Transrail ERS VAT schedule October 2023").group()
Year=re.split(r"\s","106755134 2023 October 2023 11 26 Transrail ERS VAT schedule October 2023",3)
year1=Year[1]
test=Year[2]
print(Year)

#PTIN=pd.read_excel(file_path,sheet_name="VAT schedule form--input tab",usecols="B",nrows=1,header=None,names=["Value"]).iloc[0]["Value"]

Period=pd.read_excel(file_path,sheet_name="VAT schedule form--input tab",usecols="J",nrows=1,header=None,names=["Value"]).iloc[0]["Value"]

df=pd.read_excel(file_path,sheet_name=["VAT schedule form--input tab","VAT schedule form--output tab"],header=1)

input_df=df.get("VAT schedule form--input tab")

output_df=df.get("VAT schedule form--output tab")

input_df.columns=["Date","TIN","Supplier","invoice","Description","Exempt","Zero","Unregistered","Standard_domestic","Standard_imports","Adjustment","Adjustment_other","VAT","Total"]
            
output_df.columns=["Date","TIN","Customer","invoice","Description","Exempt","Zero_exports","Zero_supplies","Standard_rated","Adjustment","Adjustment_other","VAT","Total"]

input_df["Date"]=pd.to_datetime(input_df["Date"])

output_df["Date"]=pd.to_datetime(input_df["Date"])

input_df["Exempt"]=input_df["Exempt"].astype(float)
output_df["Exempt"]=output_df["Exempt"].astype(float)
            
input_df["Zero"]=input_df["Zero"].astype(float)
output_df["Zero_exports"]=output_df["Zero_exports"].astype(float)
            
input_df["Unregistered"]=input_df["Unregistered"].astype(float)
output_df["Zero_supplies"]=output_df["Zero_supplies"].astype(float)
            
input_df["Standard_domestic"]=input_df["Standard_domestic"].astype(float)
output_df["Standard_rated"]=output_df["Standard_rated"].astype(float)
            
input_df["Standard_imports"]=input_df["Standard_imports"].astype(float)
output_df["Adjustment"]=output_df["Adjustment"].astype(float)
            
input_df["Adjustment"]=input_df["Adjustment"].astype(float)
output_df["Adjustment_other"]=output_df["Adjustment_other"].astype(float)
            
input_df["VAT"]=input_df["VAT"].astype(float)
output_df["VAT"]=output_df["VAT"].astype(float)
            
input_df["Total"]=input_df["Total"].astype(float)
output_df["Total"]=output_df["Total"].astype(float)
            
input_df["Adjustment"]=input_df["Adjustment"].astype(float)
            

input_df=input_df[input_df["Date"].notnull() & input_df["Supplier"].notnull() & input_df["invoice"].notnull()]  

output_df= output_df[output_df["Customer"].notnull() & output_df["Date"].notnull() & output_df["invoice"].notnull()]
input_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)
output_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)

#print(input_df.dtypes,output_df.dtypes)
#input_df.head(5)
input_df = input_df.fillna(value=0)
output_df = output_df.fillna(value=0)
input_df.head()

['106755134', '2023', 'October', '2023 11 26 Transrail ERS VAT schedule October 2023']


C:\Users\bedlamini\AppData\Local\Temp\ipykernel_16336\990847192.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)
C:\Users\bedlamini\AppData\Local\Temp\ipykernel_16336\990847192.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)


,Date,TIN,Supplier,invoice,Description,Exempt,Zero,Unregistered,Standard_domestic,Standard_imports,Adjustment,Adjustment_other,VAT,Total
0,2023-10-01,100257692,Swaziland Board of Executive & Trust Co Ltd,1235,Rental,0.0,0.0,0.0,10500.000000,0.0,0.0,0.0,1575.000000,12075.00
1,2023-10-02,104038776,SLS Construction,SLS037/23,sub-contractor costs,0.0,0.0,0.0,224435.756522,0.0,0.0,0.0,33665.363478,258101.12
2,2023-10-05,104997334,Mbabane Trading Company (Pty) Ltd,712789,Materials,0.0,0.0,0.0,5227.782609,0.0,0.0,0.0,784.167391,6011.95
3,2023-10-05,100124334,Eswatini Wire (Pty) Ltd,INZ05878,Materials,0.0,0.0,0.0,8685.713043,0.0,0.0,0.0,1302.856957,9988.57
4,2023-10-05,100155934,Informatics Africa PC Systems,IN095572,computer equipment,0.0,0.0,0.0,1217.391304,0.0,0.0,0.0,182.608696,1400.00


In [10]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=srv-bi-dev;'
                      'Database=ERS_data_raw;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

for index, row in input_df.iterrows():
    cursor.execute("INSERT INTO dbo.Refund_VAT_InputFormat3 (date,TIN,supplier_name,invoice,particulars,exempt,zero_rated,vat,total,ptin,created_date,unregistered,standard_rated,standard_rated_imports,Period,Adjustment,Adjustment_other) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
    row.Date, str(row.TIN), str(row.Supplier),row.invoice,row.Description,row.Exempt,row.Zero,row.VAT,row.Total,str(PTIN),today,row.Unregistered,row.Standard_domestic,row.Standard_imports,str(Period),row.Adjustment,row.Adjustment_other)

for index, row in output_df.iterrows():
    cursor.execute("INSERT INTO dbo.Refund_VATOutputFormat3 (date,TIN,customer_name,invoice,particulars,exempt,zero_rated,vat,total,ptin,CreateDate,zero_rated_supplies,standard_rated,adjustments,adjustments_other,Period) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
    row.Date, str(row.TIN), str(row.Customer),str(row.invoice),row.Description,row.Exempt,row.Zero_exports,row.VAT,row.Total,str(PTIN),today,row.Zero_supplies,row.Standard_rated,row.Adjustment,row.Adjustment_other,str(Period))
conn.commit()
cursor.close()
print("DONE")

DONE
